In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from orbit.models.lgt import LGTMAP, LGTAggregated, LGTFull
from orbit.models.dlt import DLTMAP, DLTAggregated, DLTFull
from orbit.utils.dataset import load_iclaims

from orbit.diagnostics.backtest import grid_search_orbit
from orbit.diagnostics.metrics import smape, mape, wmape

## Load Data

In [2]:
raw_data = load_iclaims()
data = raw_data.copy()

print(data.shape)
data.head(5)

(443, 7)


,week,claims,trend.unemploy,trend.filling,trend.job,sp500,vix
0,2010-01-03,13.386595,0.219882,-0.318452,0.117500,-0.417633,0.122654
1,2010-01-10,13.624218,0.219882,-0.194838,0.168794,-0.425480,0.110445
2,2010-01-17,13.398741,0.236143,-0.292477,0.117500,-0.465229,0.532339
3,2010-01-24,13.137549,0.203353,-0.194838,0.106918,-0.481751,0.428645
4,2010-01-31,13.196760,0.134360,-0.242466,0.074483,-0.488929,0.487404


## LGT Tuning Exampe

In [3]:
lgt = LGTMAP(
    date_col='week',
    response_col='claims',
    seasonality=52,
)

In [4]:
param_grid = {
    'level_sm_input': [0.3, 0.5, 0.8],  
    'seasonality_sm_input': [0.3, 0.5, 0.8],
}

In [5]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=lgt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.3}


 11%|█         | 1/9 [00:04<00:36,  4.62s/it]

tuning metric:0.0066442
tuning hyper-params {'level_sm_input': 0.3, 'seasonality_sm_input': 0.5}


 22%|██▏       | 2/9 [00:08<00:29,  4.19s/it]

...

tuning metric:0.0093521
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.5}


 89%|████████▉ | 8/9 [00:32<00:04,  4.10s/it]

tuning metric:0.0093154
tuning hyper-params {'level_sm_input': 0.8, 'seasonality_sm_input': 0.8}


100%|██████████| 9/9 [00:36<00:00,  4.10s/it]

tuning metric:0.0079935


In [6]:
tuned_df.head()

,level_sm_input,seasonality_sm_input,metrics
0,0.3,0.3,0.006644
1,0.3,0.5,0.006039
2,0.3,0.8,0.005653
3,0.5,0.3,0.007682
4,0.5,0.5,0.006923


In [7]:
best_params

[{'level_sm_input': 0.3, 'seasonality_sm_input': 0.8}]

## DLT Tuning Example

In [8]:
dlt = DLTMAP(
    date_col='week',
    response_col='claims',
    regressor_col=['trend.unemploy', 'trend.filling', 'trend.job'],
    seasonality=52,
)

In [9]:
param_grid = {
    'damped_factor': [0.3, 0.5, 0.8], 
    'slope_sm_input': [0.3, 0.5, 0.8],
}

In [10]:
best_params, tuned_df = grid_search_orbit(param_grid, 
                                        model=dlt, 
                                        df=data,
                                        min_train_len=100, incremental_len=100, forecast_len=20, 
                                        metrics=None, criteria=None, verbose=True)

  0%|          | 0/9 [00:00<?, ?it/s]

tuning hyper-params {'damped_factor': 0.3, 'slope_sm_input': 0.3}


 11%|█         | 1/9 [00:04<00:35,  4.42s/it]

tuning metric:0.0057861
tuning hyper-params {'damped_factor': 0.3, 'slope_sm_input': 0.5}


 22%|██▏       | 2/9 [00:08<00:29,  4.24s/it]

...

tuning metric:0.0057157
tuning hyper-params {'damped_factor': 0.8, 'slope_sm_input': 0.5}


 89%|████████▉ | 8/9 [00:33<00:04,  4.19s/it]

tuning metric:0.0057012
tuning hyper-params {'damped_factor': 0.8, 'slope_sm_input': 0.8}


100%|██████████| 9/9 [00:37<00:00,  4.19s/it]

tuning metric:0.0057447


In [11]:
tuned_df.head()

,damped_factor,slope_sm_input,metrics
0,0.3,0.3,0.005786
1,0.3,0.5,0.005656
2,0.3,0.8,0.005644
3,0.5,0.3,0.005744
4,0.5,0.5,0.005605


In [12]:
best_params

[{'damped_factor': 0.5, 'slope_sm_input': 0.8}]